# Analiza pomiarów

Analiza i wizualizacja danych zebranych przy uruchamianiu algorytmu Knutha oraz algorytmu ewolucyjnego z różnymi parametrami.

In [6]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

## Algorytm Knutha

In [7]:
# TODO: Jakaś prosta analiza jedych wyników

## Algorytm gentyczny

Analizy wyników dla różny parametrów.